In [ ]:
import scipy.io
import numpy as np
from sklearn.metrics import auc
import pingouin as pg
import pandas as pd
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.anova import AnovaRM

In [ ]:
hc_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/atlas_BNA/BNA_240_flipped_N32_retroicor_SBB4_prop_bin_window/HC/CharPath240_binwin_HC.mat'
st_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/atlas_BNA/BNA_240_flipped_N32_retroicor_SBB4_prop_bin_window/ST/CharPath240_binwin.mat'

In [ ]:
minimum_connectivity_threshold = 0.3

In [ ]:
hc_data_mat = scipy.io.loadmat(hc_data_path)
st_data_mat = scipy.io.loadmat(st_data_path)

In [ ]:
# converting matlab mat to arrays
# transposing to have shape (n_subj, n_thresholds)
hc_glob_eff = np.vstack(hc_data_mat['GlobEfficiency'][0][0]).T
st0_glob_eff = np.vstack(np.vstack(st_data_mat['GlobEfficiency'][0][0])[0][0]).T
st1_glob_eff = np.vstack(np.vstack(st_data_mat['GlobEfficiency'][0][0])[1][0]).T
st2_glob_eff = np.vstack(np.vstack(st_data_mat['GlobEfficiency'][0][0])[2][0]).T

In [ ]:
# correct for missing values - (pt1 TP1, pt5 Tp2, Pt13 (=pt17) TP3)
n_bins = st0_glob_eff.shape[1]
st0_glob_eff = np.insert(st0_glob_eff, 0, np.full((n_bins), np.NaN), axis=0)
st1_glob_eff = np.insert(st1_glob_eff, 4, np.full((n_bins), np.NaN), axis=0)
st2_glob_eff = np.insert(st2_glob_eff, 12, np.full((n_bins), np.NaN), axis=0)

In [ ]:
n_hc = hc_glob_eff.shape[0]
n_st = st0_glob_eff.shape[0]

In [ ]:
# only analyse thresholds above minimum_connectivity_threshold
minimum_connectivity_threshold_index = int(minimum_connectivity_threshold*10 - 1)
connectivity_thresholds = np.arange(minimum_connectivity_threshold, 1.1, 0.1)

In [ ]:
def glob_eff_auc_per_subject(subjects_glob_eff, connectivity_thresholds, minimum_connectivity_threshold_index):
    # compute global efficiency AUC over connectivity_thresholds for each subject
    subjects_glob_eff_auc = []
    for subj_idx in range(subjects_glob_eff.shape[0]):
        subjects_glob_eff_auc.append(
            auc(connectivity_thresholds,
                subjects_glob_eff[subj_idx, minimum_connectivity_threshold_index:])
        )
    return subjects_glob_eff_auc

In [ ]:
hc_glob_eff_auc = glob_eff_auc_per_subject(hc_glob_eff, connectivity_thresholds, minimum_connectivity_threshold_index)
st0_glob_eff_auc = glob_eff_auc_per_subject(st0_glob_eff, connectivity_thresholds, minimum_connectivity_threshold_index)
st1_glob_eff_auc = glob_eff_auc_per_subject(st1_glob_eff, connectivity_thresholds, minimum_connectivity_threshold_index)
st2_glob_eff_auc = glob_eff_auc_per_subject(st2_glob_eff, connectivity_thresholds, minimum_connectivity_threshold_index)

In [ ]:
# Compare to healthy subjects using one-way ANOVA
f_oneway(hc_glob_eff_auc, st0_glob_eff_auc, st1_glob_eff_auc, st2_glob_eff_auc)


In [ ]:
# glob_eff_auc_df = pd.DataFrame(hc_glob_eff_auc, st0_glob_eff_auc, st1_glob_eff_auc, st2_glob_eff_auc,
#                                columns=['hc', 'st0', 'st1', 'st2'])
all_subj_idx = np.arange(n_hc + n_st)
hc_idx = all_subj_idx[:n_hc]
st_idx = all_subj_idx[n_hc:]

columns = ['subject', 'glob_eff_auc', 'group', 'timepoint']
hc_glob_eff_auc_df = pd.DataFrame([hc_idx, hc_glob_eff_auc, np.repeat('hc', n_hc), np.repeat(0, n_hc)]).T
st0_glob_eff_auc_df = pd.DataFrame([st_idx, st0_glob_eff_auc, np.repeat('st', n_st), np.repeat(0, n_st)]).T
st1_glob_eff_auc_df = pd.DataFrame([st_idx, st1_glob_eff_auc, np.repeat('st', n_st), np.repeat(1, n_st)]).T
st2_glob_eff_auc_df = pd.DataFrame([st_idx, st2_glob_eff_auc, np.repeat('st', n_st), np.repeat(2, n_st)]).T

glob_eff_auc_df = pd.concat([hc_glob_eff_auc_df,st0_glob_eff_auc_df, st1_glob_eff_auc_df, st2_glob_eff_auc_df],
                            ignore_index=True)
all_st_glob_eff_auc_df = pd.concat([st0_glob_eff_auc_df, st1_glob_eff_auc_df, st2_glob_eff_auc_df],
                            ignore_index=True)
glob_eff_auc_df.columns = columns
all_st_glob_eff_auc_df.columns = columns

# convert auc to numeric
glob_eff_auc_df['glob_eff_auc'] = glob_eff_auc_df['glob_eff_auc'].astype('float')
all_st_glob_eff_auc_df['glob_eff_auc'] = all_st_glob_eff_auc_df['glob_eff_auc'].astype('float')

# convert timepoint to numeric
glob_eff_auc_df['timepoint'] = glob_eff_auc_df['timepoint'].astype('float')
all_st_glob_eff_auc_df['timepoint'] = all_st_glob_eff_auc_df['timepoint'].astype('float')


glob_eff_auc_df.head()

In [ ]:
# without AUC data reduction

hc_glob_eff_df = pd.DataFrame(data=hc_glob_eff)
hc_glob_eff_df['subject'] = hc_idx
hc_glob_eff_df = hc_glob_eff_df.melt(id_vars=['subject'], var_name='density_bin', value_name='glob_eff')
hc_glob_eff_df['timepoint'] = 0
hc_glob_eff_df['group'] = 'hc'

st0_glob_eff_df = pd.DataFrame(data=st0_glob_eff)
st0_glob_eff_df['subject'] = st_idx
st0_glob_eff_df = st0_glob_eff_df.melt(id_vars=['subject'], var_name='density_bin', value_name='glob_eff')
st0_glob_eff_df['timepoint'] = 0
st0_glob_eff_df['group'] = 'st'

st1_glob_eff_df = pd.DataFrame(data=st1_glob_eff)
st1_glob_eff_df['subject'] = st_idx
st1_glob_eff_df = st1_glob_eff_df.melt(id_vars=['subject'], var_name='density_bin', value_name='glob_eff')
st1_glob_eff_df['timepoint'] = 1
st1_glob_eff_df['group'] = 'st'

st2_glob_eff_df = pd.DataFrame(data=st2_glob_eff)
st2_glob_eff_df['subject'] = st_idx
st2_glob_eff_df = st2_glob_eff_df.melt(id_vars=['subject'], var_name='density_bin', value_name='glob_eff')
st2_glob_eff_df['timepoint'] = 2
st2_glob_eff_df['group'] = 'st'

glob_eff_df = pd.concat([hc_glob_eff_df,st0_glob_eff_df, st1_glob_eff_df, st2_glob_eff_df],
                            ignore_index=True)

In [ ]:
# convert auc to numeric
glob_eff_df['glob_eff'] = glob_eff_df['glob_eff'].astype('float')
glob_eff_df['density_bin'] = glob_eff_df['density_bin'].astype('float')

In [ ]:
# make density bins correspond to initial nomenclature (1-10)
glob_eff_df['density_bin'] += 1

# threshold bins at [0.3-1]
glob_eff_df = glob_eff_df[glob_eff_df['density_bin'] >= 3]

In [ ]:
ax = sns.boxplot(x="timepoint", y="glob_eff_auc", hue="group", data=glob_eff_auc_df, palette="Set3")
ax.set_title('Global Efficiency AUC')
plt.show()

In [ ]:
ax = sns.boxplot(x="timepoint", y="glob_eff", hue="group", data=glob_eff_df, palette="Set3")
ax.set_title('Global Efficiency')
plt.show()

In [ ]:
pg.rm_anova(data=glob_eff_auc_df, dv='glob_eff_auc', subject='subject', within=['group'], detailed=True)


In [ ]:
pg.rm_anova(data=all_st_glob_eff_auc_df, dv='glob_eff_auc', subject='subject', within=['timepoint'], detailed=True)


In [ ]:
aov = pg.mixed_anova(dv='glob_eff_auc', within='timepoint', between='group', subject='subject', data=glob_eff_auc_df)
# Pretty printing of ANOVA summary
pg.print_table(aov)

In [ ]:
# # conduct ANOVA using mixedlm
# my_model_fit = smf.mixedlm("glob_eff_auc ~ timepoint", glob_eff_auc_df, groups=glob_eff_auc_df["group"]).fit()
# # get random effects
# my_model_fit.random_effects
# # get fixed effects
# my_model_fit.summary()

In [ ]:
glob_eff_auc_df.to_csv('glob_eff_auc_df.csv', index=False)

In [ ]:
glob_eff_df.to_csv('glob_eff_df.csv', index=False)